In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

# Define the generator
def build_generator(latent_dim):
    model = keras.Sequential()
    model.add(layers.Dense(256, input_dim=latent_dim))
    model.add(layers.LeakyReLU(alpha=0.2))
    model.add(layers.BatchNormalization(momentum=0.8))
    model.add(layers.Dense(512))
    model.add(layers.LeakyReLU(alpha=0.2))
    model.add(layers.BatchNormalization(momentum=0.8))
    model.add(layers.Dense(1024))
    model.add(layers.LeakyReLU(alpha=0.2))
    model.add(layers.BatchNormalization(momentum=0.8))
    model.add(layers.Dense(784, activation='tanh'))
    model.add(layers.Reshape((28, 28, 1)))
    return model

# Define the discriminator
def build_discriminator(image_shape):
    model = keras.Sequential()
    model.add(layers.Flatten(input_shape=image_shape))
    model.add(layers.Dense(1024))
    model.add(layers.LeakyReLU(alpha=0.2))
    model.add(layers.Dense(512))
    model.add(layers.LeakyReLU(alpha=0.2))
    model.add(layers.Dense(256))
    model.add(layers.LeakyReLU(alpha=0.2))
    model.add(layers.Dense(1, activation='sigmoid'))
    return model

# Create the GAN
def build_gan(generator, discriminator):
    discriminator.trainable = False
    model = keras.Sequential()
    model.add(generator)
    model.add(discriminator)
    return model

# Define GAN parameters
latent_dim = 100
image_shape = (28, 28, 1)

# Build and compile the models
generator = build_generator(latent_dim)
discriminator = build_discriminator(image_shape)
discriminator.compile(loss='binary_crossentropy', optimizer=keras.optimizers.Adam(0.0002, 0.5))
gan = build_gan(generator, discriminator)
gan.compile(loss='binary_crossentropy', optimizer=keras.optimizers.Adam(0.0002, 0.5))

# Training loop goes here

In [ ]:
import numpy as np


In [ ]:
# Import your dataset and preprocess it as needed

# Training parameters
batch_size = 64
epochs = 20000

# Training loop
for epoch in range(epochs):
    # ---------------------
    #  Train the Discriminator
    # ---------------------

    # Select a random batch of real images
    idx = np.random.randint(0, X_train.shape[0], batch_size)
    real_images = X_train[idx]

    # Generate a batch of fake images
    noise = np.random.normal(0, 1, (batch_size, latent_dim))
    fake_images = generator.predict(noise)

    # Labels for the real and fake images
    real_labels = np.ones((batch_size, 1))
    fake_labels = np.zeros((batch_size, 1))

    # Train the discriminator
    d_loss_real = discriminator.train_on_batch(real_images, real_labels)
    d_loss_fake = discriminator.train_on_batch(fake_images, fake_labels)
    d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)

    # ---------------------
    #  Train the Generator
    # ---------------------

    # Generate a batch of noise
    noise = np.random.normal(0, 1, (batch_size, latent_dim))

    # Labels for the generator
    valid_labels = np.ones((batch_size, 1))

    # Train the generator
    g_loss = gan.train_on_batch(noise, valid_labels)

    # Print the progress
    print(f"Epoch {epoch}/{epochs}, D Loss: {d_loss[0]}, G Loss: {g_loss}")

    # Optionally, you can save generated images at specific intervals

# After training, you can save the generator model for future use
generator.save("gan_generator.h5")